In [1]:
#pip install geopy
#pip install bs4
#pip install requests

In [2]:
import os
import numpy as np
import pandas as pd
import re
from geopy.point import Point
from geopy.geocoders import Nominatim
import requests
from bs4 import BeautifulSoup

In [3]:
os.getcwd()

'/Users/jbrable/Documents/ENSAE_2AD/S1/python/UFO'

In [4]:
df = pd.read_csv("data/ufo_sightings_scrubbed.csv")

/var/folders/j3/7p5c8ygd7yg6bgkb6r90ddhm0000gn/T/ipykernel_35772/2315484744.py:1: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/ufo_sightings_scrubbed.csv")


In [5]:
df.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,2004-04-27,29.8830556,-97.941111
1,1949-10-10 21:00:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.38421,-98.581082
2,1955-10-10 17:00:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.2,-2.916667
3,1956-10-10 21:00:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.9783333,-96.645833
4,1960-10-10 20:00:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.4180556,-157.803611


In [6]:
df.shape

(80332, 11)

# 1) Cleaning data

## 1.1) Column names

In [7]:
df.columns

Index(['datetime', 'city', 'state', 'country', 'shape', 'duration (seconds)',
       'duration (hours/min)', 'comments', 'date posted', 'latitude',
       'longitude '],
      dtype='object')

We notice a first problem, that is, the 'longitude' column label is mispelled : it contains a blank space at the end of the string. Let's remove it in order to avoid further issues:

In [8]:
df = df.rename(columns = {'longitude ': 'longitude'})
#df.columns

In [9]:
df.dtypes

datetime                 object
city                     object
state                    object
country                  object
shape                    object
duration (seconds)       object
duration (hours/min)     object
comments                 object
date posted              object
latitude                 object
longitude               float64
dtype: object

We also notice that 'longitude' column is a type float, which is pretty logical, whereas 'latitude' column if typed as an object : why ? Let's check it out:

In [10]:
df[df['latitude'].astype(str).str.contains('[a-zA-Z]')]

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
43782,1974-05-22 05:30:00,mescalero indian reservation,nm,NaN,rectangle,180,two hours,Huge rectangular object emmitting intense whit...,2012-04-18,33q.200088,-105.624152


We see that this problem is probably due to a typo: the corresponding latitude for 'mescalero indian reservation' contains a letter ('q'). Since this issue only concerns a single row, we can check the real latitude of this place on the Internet (if there were too many rows, we would have chosen another soluton). It turns out that the real latitude is 33.2 so the error was indeed the result of a typo, so we can remove the misplaced letter:

In [11]:
df['latitude'] = pd.to_numeric(df['latitude'].astype(str).str.extract('(\d+.\d+|\d+)', expand = False))
#df.dtypes #now both of the 'latitude' and 'longitude' columns have the right type!

## 1.2) Handling NaN values

In [12]:
def count_na(df: pd.DataFrame) -> pd.DataFrame:
    
    """Returns the sum and percentage of NaN values for each column of a given dataframe"""
    
    return pd.DataFrame({'sum': df.isna().sum(),
                         'perc': df.isna().sum() * 100 / len(df)}).sort_values(by='perc',
                                                                               ascending=False)

In [13]:
count_na(df)

,sum,perc
country,9670,12.037544
state,5797,7.216302
shape,1932,2.405019
comments,15,0.018673
datetime,0,0.000000
city,0,0.000000
duration (seconds),0,0.000000
duration (hours/min),0,0.000000
date posted,0,0.000000
latitude,0,0.000000


We notice that there are quite a lot of NaN values in both "country" and "state" columns. Let's see a few rows that are concerned by this phenomenon:

In [14]:
df.query('country.isna() | state.isna()', engine = 'python')

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
1,1949-10-10 21:00:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082
2,1955-10-10 17:00:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,-2.916667
6,1965-10-10 21:00:00,penarth (uk/wales),NaN,gb,circle,180,about 3 mins,penarth uk circle 3mins stayed 30ft above m...,2006-02-14,51.434722,-3.180000
18,1973-10-10 23:00:00,bermuda nas,NaN,NaN,light,20,20 sec.,saw fast moving blip on the radar scope thin w...,2002-01-11,32.364167,-64.678611
20,1974-10-10 21:30:00,cardiff (uk/wales),NaN,gb,disk,1200,20 minutes,back in 1974 I was 19 at the time and lived i...,2007-02-01,51.500000,-3.200000
...,...,...,...,...,...,...,...,...,...,...,...
80254,2009-09-09 21:15:00,nottinghamshire (uk/england),NaN,gb,fireball,600.0,10 mins,resembled orange flame imagine a transparent h...,2009-12-12,53.166667,-1.000000
80255,2009-09-09 21:38:00,kaiserlautern (germany),NaN,de,light,40.0,about 40 seconds,2 white lights over Kaiserslautern&#44 ramstei...,2009-12-12,49.450000,7.750000
80319,2013-09-09 20:15:00,clifton,nj,NaN,other,3600.0,~1hr+,Luminous line seen in New Jersey sky.,2013-09-30,40.858433,-74.163755
80322,2013-09-09 21:00:00,aleksandrow (poland),NaN,NaN,light,15.0,15 seconds,Two points of light following one another in a...,2013-09-30,50.465843,22.891814


* For the "state" column, the problem is likely due to the fact that a "state" is commonly used in the US or in Canada, but not in other countries. Thus, we won't touch at this column for the moment.

### 1.2.1 Cleaning "country" column

In a first approach, we decided to fill NaN values in the "country" column by searching the corresponding country of the given latitude and longitude (this two columns are always filled). To do so, we will use the "geopy" library (geopy is actually a client), which allows to find the country, based on the geographic coordinates.

In [15]:
# connecting to the service
geolocator = Nominatim(user_agent="begining")

In [16]:
# partioning our df between missing values and non-missing values
df1 = df[df['country'].isna()].reset_index(drop=False)
df2 = df[~df['country'].isna()]

In [17]:
df1.shape[0] + df2.shape[0] == df.shape[0]

True

In [18]:
def to_point(lat: float,lon: float):
    
    """Transforms the given latitude and longitude into a coordinate system (use Point method
    from geopy"""
    
    return Point(lat, lon)

In [19]:
# getting a full coordinate system for each row, based on the latitude and longitude column
df1['coords'] = [to_point(x, y) for x, y in zip(df1['latitude'], df1['longitude'])]

In [20]:
def get_geo_feature(coord, feature: str):
    
    """Get the desired feature from a given geographic coordinate (e.g.,
    country name, country code, etc: please refers to the geopy doc for more informations)"""
    
    try:
        location = geolocator.reverse(coord)
        return location.raw['address'][feature]
    
    except:
        return np.nan

We will apply our functions "get_geo_feature" to obtain the country code for each row where the country is missing. **This step might be time costly (approx. 1h30).**

In [21]:
df1['country'] = np.vectorize(get_geo_feature)(df1['coords'].values, 'country_code').tolist()

In [25]:
df1['country'] = np.where(df1['country']=='nan', np.nan, df1['country'])

In [30]:
df1['country'].isna().sum()

592

In [33]:
df1 = df1.drop(['coords', 'index'], axis=1)

The observations where the country was (formerly) missing must be combined with those where the country was known. Thus, we have to binds the rows of the two dataframes in order to get al the observations reunited:

In [35]:
df = pd.concat([df1, df2], ignore_index=True, axis=0)

In [36]:
df.shape # we got back the shape of the begnining

(80332, 11)

In a second approach, we will use the information that we have in the "city" column: indeed we notice that there is some potential information about the country inside this column (see above)!! But sometimes, this information is not relevant, for example if we look at the tail of the dataframe above, "hollyridge" is clearny not a country, but a US state! We have to make sure that if there is additionnal information in the "city", this information is actually a country!

The easiest way to do this is to use web-scraping to get the list of all countries with the corresponding country codes (country codes will be useful to standardize the country column):

In [37]:
# Creating the parser
URL = "https://www.iban.com/country-codes"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

In [38]:
for table in soup.find_all('table'):
    print(table.get('class'))
    
table = soup.find('table', class_='table table-bordered downloads tablesorter')
#print(table)

['table', 'table-bordered', 'downloads', 'tablesorter']


In [39]:
countries_id = pd.DataFrame(columns=['Country', 'Alpha-2 code'])
              
# collecting country names and country codes
for row in table.tbody.find_all('tr'):
     # STEPS FOR ONE ROW :
    
    # find all data for each column
    columns = row.find_all('td')
    
    # if the row is not empty, we get the 2 informations that we want, ie country and alpha-2 code
    if(columns != []):
        country = columns[0].text.strip().lower()
        alpha_2_code = columns[1].text.strip().lower()
        
        # then, we merge this 2 informations together
        df_to_append = pd.DataFrame({'Country': country,
                                     'Alpha-2 code': alpha_2_code},
                                   index=[0])
        
        # and finally append to the "main" dataframe
        countries_id = pd.concat([countries_id, df_to_append], ignore_index=True)
        
print(countries_id.shape)
display(countries_id.head())

(249, 2)


,Country,Alpha-2 code
0,afghanistan,af
1,åland islands,ax
2,albania,al
3,algeria,dz
4,american samoa,as


Now that we have a list of all the countries in the world, we can first standardize the country column by converting the country codes into country names, for non-missing country values in our dataframe. To do so, we just have to create a dictionnary with both the country-code and the corresponding country name, and then map it to the "country" column:

In [40]:
countries_dict = dict(zip(countries_id['Alpha-2 code'], countries_id['Country']))

In [42]:
df['country'] = df['country'].replace(countries_dict)

Finally, we can easily detect if there is some informative content in the "city" column for each row, by comparing it  with the list of the countries that we previously built. In order to perform this step, we will have to use regex: first, we create a "large pattern" with our country Series:

In [43]:
pattern_countries = '('+ '|'.join(countries_id['Country'].apply(lambda x: re.sub(r'\((.*)\)', '', x))) +')'
#pattern_countries

Then, we match this pattern with the "city" column, for each row (NB: str.extract method is vectorized):

In [44]:
df['test'] = df['city'].str.extract(pattern_countries)[0]

In [48]:
df.country.isna().sum()

592

Then, we can replace the values of the country that are missing by the values obtained with our method:

In [49]:
df['country'] = np.where(df['country'].isna(), df['test'], df['country'])

In [50]:
print(df[df['country'].isna()].shape)
print(df[df['test'].isna()].shape)
print(df[df['country'].isna() & df['test'].isna()].shape)

(199, 12)
(73154, 12)
(199, 12)


We see that our method allowed to reduce the number of NaN values in the "country" column. But there is still some missing values: let's check those rows to see the issue:

In [51]:
pd.set_option('display.max_rows', None)

print(df[df['test'].isna() & df['country'].isna()]['city'].value_counts().sort_index())

pd.reset_option('display.max_rows')

arkansas (eastern; on us hwy 70 abt 75 m w/o memphis&#44 tn)     1
arkansas (while driving)                                         1
armidale (australi)                                              1
atlantic ocean                                                  11
atlantic ocean (at sea)                                          1
atlantic ocean (between nassau&#44 bahamas&#44 and florida)      1
atlantic ocean (between san juan and u.s. virgin isl.)           1
atlantic ocean (cv-67 u.s.navy)                                  1
atlantic ocean (in flight)                                       2
atlantic ocean (inflight)                                        1
atlantic ocean (mid-atlantic)                                    1
atlantic ocean (middle)                                          1
atlantic ocean (nc/250+nm off the coast)                         1
atlantic ocean (off africa)                                      1
atlantic ocean (on cruise ship)                               

We notice 2 main issues:
* the "city" column may contain a sea or ocean name. For these rows, we are going to define the country as "sea" or "ocean";
* there is some issues with country names such as "brazil", "bolivia", "australia". For what is possible and easy, we will replace the country names by hand. But for the remaining rows (there aren't many), we will put a NaN value.

In [84]:
def replace_exceptions(row: str) -> str or float:
    
    """This function was made to be applied to a single dataframe string column. Return either 'ocean',
    'sea', or a predefined country name."""
    
    if re.search(r"ocean|sea", row):
        return re.findall(r'ocean|sea', row)[0]
    
    elif re.search(r"brasil", row):
        return "brazil"
    
    elif re.search(r"uk/england", row):
        return "united kingdom of great britain and northern ireland (the)"
    
    elif re.search(r"netherlands", row):
        return "netherlands (the)"
    
    elif re.search(r"bolivia", row):
        return "bolivia (plurinational state of)"
    
    elif re.search(r"australi", row):
        return "australia"
    
    else:
        return np.nan

#pbb['test'] = pbb['city'].apply(replace_exceptions)
#pbb[pbb['test'].isna()]

In [89]:
df[(df['country'].isna())].shape

(199, 13)

In [91]:
df['country'] = np.where((df['test'].isna() & df['country'].isna()), df['city'].apply(replace_exceptions), df['country'])

In [92]:
df[(df['country'].isna())].shape

(112, 13)

For the country name that are still missing, we decided to delete them, because they are very few of them, we decided to let them as NaN values.

## 1.3) Cleaning 'city' column

If we have a closer look to this column, we see that the values are not harmonized and can be confusing and noisy:

In [93]:
df.query("city.str.contains('\(') | city.str.contains('\)')", engine='python')

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,test,test2
2,1979-10-10 22:00:00,saddle lake (canada),ab,canada,triangle,270,4.5 or more min.,Lights far above&#44 that glance; then flee f...,2005-01-19,53.970571,-111.689885,canada,canada
3,1982-10-10 07:00:00,gisborne (new zealand),NaN,new zealand,disk,120,2min,gisborne nz 1982 wainui beach to sponge bay,2002-01-11,38.662334,178.017649,new zealand,new zealand
6,1993-10-10 03:00:00,zlatoust (russia),NaN,russian federation (the),sphere,1200,20 minutes,I woke up at night and looked out the window n...,2004-12-14,55.183333,59.650000,NaN,russian federation (the)
8,1996-10-10 20:00:00,lake macquarie (nsw&#44 australia),NaN,australia,light,300,5 min,RED LIGHT WITH OTHER RED FLASHING LIGHT&#44 ON...,1999-05-24,33.093373,151.588982,australia,australia
10,1999-10-10 22:30:00,casey key (north end of),fl,united states of america (the),triangle,120,several minutes,A large trianglual shaped craft flew from hori...,2002-09-19,27.150053,-82.480653,NaN,united states of america (the)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80298,2012-09-09 14:00:00,pictou (canada),ns,canada,disk,600.0,10 minutes,Spotted a object over the town&#39s business d...,2012-09-24,45.666667,-62.700000,canada,canada
80311,2012-09-09 21:00:00,new york city (brooklyn),ny,united states of america (the),light,1290.0,21:30,Glowing&#44 circular lights visible in the clo...,2012-09-24,40.714167,-74.006389,NaN,united states of america (the)
80317,2013-09-09 01:50:00,buffalo (west of; on highway 90 west),ny,united states of america (the),triangle,180.0,3 minutes,Massive Flat Black triangle with 3 red lights.,2013-09-30,42.886389,-78.878611,NaN,united states of america (the)
80321,2013-09-09 13:10:00,calmar (canada),ab,canada,unknown,90.0,45-90 seconds,Fastest dot I have ever seen in the sky&#33,2013-09-09,53.250000,-113.783333,canada,canada


We have to remove all parenthesis from this column. Then, we have to harmonize the city name; and we will eventually standardize this column by capitalizing the city names.

In [94]:
def print_city_unique():
    return print(len(set(df['city'])))

In [95]:
def clean_city_col(df: pd.DataFrame) -> pd.DataFrame:
    
    print_city_unique()
    
    # removing parentheses and their content for the column "city"
    df['city'] = df['city'].str.replace(r"\(.*\)","", regex=True)
    print_city_unique()
    
    # removing non-alphanumeric characters from the strings for the column "city", including numbers
    df['city'] = [re.sub('[^\w\s]|\d+', '', x) for x in df['city']]
    print_city_unique()
    
    # turning the first letter of each city name into a capital letter
    df['city'] = df['city'].str.title()
    print_city_unique()
    
    return df

In [96]:
df = clean_city_col(df)
# the number of differents city values at each step of the processing is decreasing: perfect

19900
18386
18290
18290


Our processing of the city column has resulted in the harmonization and standardization of city names. This achievement will be very convenient for the interactive map that we are going to create and also for an in-depth statistical analysis.

In [103]:
df = df.drop(['test', 'test2'], axis=1)

## 1.4) Cleaning "shape" column

In [105]:
df["shape"].value_counts()

light        16565
triangle      7865
circle        7608
fireball      6208
other         5649
unknown       5584
sphere        5387
disk          5213
oval          3733
formation     2457
cigar         2057
changing      1962
flash         1328
rectangle     1297
cylinder      1283
diamond       1178
chevron        952
egg            759
teardrop       750
cone           316
cross          233
delta            7
crescent         2
round            2
dome             1
pyramid          1
flare            1
hexagon          1
changed          1
Name: shape, dtype: int64

## next###

In [8]:
df["datetime"].sort_values(ascending = False)

46302    2014-05-08 18:45:00
46301    2014-05-08 00:00:00
46155    2014-05-07 23:30:00
46154    2014-05-07 21:20:00
46153    2014-05-07 21:10:00
                ...         
46694    1920-06-11 21:00:00
40275    1916-04-05 13:00:00
46858    1910-06-01 15:00:00
10580    1910-01-02 00:00:00
7863     1906-11-11 00:00:00
Name: datetime, Length: 80332, dtype: object

In [9]:
df["comments"].head()

0    This event took place in early fall around 194...
1    1949 Lackland AFB&#44 TX.  Lights racing acros...
2    Green/Orange circular disc over Chester&#44 En...
3    My older brother and twin sister were leaving ...
4    AS a Marine 1st Lt. flying an FJ4B fighter/att...
Name: comments, dtype: object

# 2) Data Exportation

In [106]:
count_na(df)

,sum,perc
state,5797,7.216302
shape,1932,2.405019
country,112,0.139421
comments,15,0.018673
datetime,0,0.000000
city,0,0.000000
duration (seconds),0,0.000000
duration (hours/min),0,0.000000
date posted,0,0.000000
latitude,0,0.000000


Since we have clean our data, we should export it into a new .csv file, so as to facilitate oru collaborative work:

In [392]:
df.to_csv("data/ufo_sightings_clean.csv", index = False)